In [2]:
import pandas as pd
import requests

## Hello World

In [5]:
requests.get("https://api.chess.com/pub/player/uneatenbean/stats").json()

{'chess_daily': {'last': {'rating': 796, 'date': 1642775141, 'rd': 156},
  'best': {'rating': 796,
   'date': 1642775141,
   'game': 'https://www.chess.com/game/daily/377756365'},
  'record': {'win': 5,
   'loss': 0,
   'draw': 0,
   'time_per_move': 11748,
   'timeout_percent': 0}},
 'chess_rapid': {'last': {'rating': 1282, 'date': 1656591197, 'rd': 51},
  'best': {'rating': 1360,
   'date': 1651775655,
   'game': 'https://www.chess.com/game/live/44785791809'},
  'record': {'win': 188, 'loss': 103, 'draw': 7}},
 'chess_bullet': {'last': {'rating': 1302, 'date': 1656856967, 'rd': 26},
  'best': {'rating': 1302,
   'date': 1656856967,
   'game': 'https://www.chess.com/game/live/50595572023'},
  'record': {'win': 425, 'loss': 317, 'draw': 9}},
 'chess_blitz': {'last': {'rating': 1109, 'date': 1656886798, 'rd': 40},
  'best': {'rating': 1180,
   'date': 1649669695,
   'game': 'https://www.chess.com/game/live/43407601701'},
  'record': {'win': 473, 'loss': 362, 'draw': 30}},
 'fide': 0,
 '

## Games

In [13]:
from chess import pgn
import io

In [16]:
games_res = requests.get("https://api.chess.com/pub/player/uneatenbean/games/2022/06/pgn").text

In [17]:
game = pgn.read_game(io.StringIO(games_res))

In [18]:
game.headers

Headers(Event='Live Chess', Site='Chess.com', Date='2022.06.30', Round='-', White='EpicAnon', Black='uneatenbean', Result='1-0', BlackElo='1212', CurrentPosition='6k1/2r1bppp/Q3b3/1B1p4/3P4/4P3/PP3PPP/4R1K1 w - -', ECO='D35', ECOUrl='https://www.chess.com/openings/Queens-Gambit-Declined-Harrwitz-Attack', EndDate='2022.06.30', EndTime='21:52:16', Link='https://www.chess.com/game/live/50364643009', StartTime='21:51:17', Termination='EpicAnon won by resignation', TimeControl='60', Timezone='UTC', UTCDate='2022.06.30', UTCTime='21:51:17', WhiteElo='1193')

**TimeControl** is in seconds, I will have to filter time control for common Rapid values

**BlackElo; WhiteElo**: I can get the elo at the start of every game, which should be a sufficient marker

### Multiple games

In [31]:
games_list = games_res.split("\n\n\n")
len(games_list)
parsed_games = [pgn.read_game(io.StringIO(g)) for g in games_list]

In [ ]:
[pgn.read_game(io.StringIO(g)).headers["TimeControl"] for g in games_list]

### TimeControl

In [25]:
import re

In [45]:
def isRapid(time_control: str) -> bool:
    match = re.match("\d+", time_control)
    if not match: raise "Invalid Time Control"
    return int(match.group(0)) >= 600

assert isRapid("600")
assert not isRapid("100")

def isLive(event: str) -> bool:
    return event == "Live Chess"

In [46]:
rapid_games = list(filter(lambda g: isRapid(g.headers["TimeControl"]) and isLive(g.headers["Event"]), parsed_games))

In [48]:
print(len(rapid_games))
[print(g.headers["White"], " - ", g.headers["Black"], " @ ", g.headers["Date"]) for g in rapid_games]

17
uneatenbean  -  lucaippolito  @  2022.06.30
Forchezque  -  uneatenbean  @  2022.06.30
uneatenbean  -  Zabara77  @  2022.06.30
lashenoOo  -  uneatenbean  @  2022.06.30
shuweii  -  uneatenbean  @  2022.06.29
uneatenbean  -  shuweii  @  2022.06.29
shuweii  -  uneatenbean  @  2022.06.29
uneatenbean  -  shuweii  @  2022.06.29
mostafahusseinsalem  -  uneatenbean  @  2022.06.23
uneatenbean  -  mosheassous10  @  2022.06.23
vulia  -  uneatenbean  @  2022.06.23
uneatenbean  -  RockyFromage  @  2022.06.23
uneatenbean  -  AnnoyingShip  @  2022.06.23
4105181a  -  uneatenbean  @  2022.06.23
uneatenbean  -  sethindonesia  @  2022.06.04
uneatenbean  -  RicardoCampos19  @  2022.06.04
uneatenbean  -  floydTO  @  2022.06.04


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

## Graphs
First, I want rating over time

In [50]:
import matplotlib.pyplot as plt

In [49]:
USERNAME = "uneatenbean"

In [51]:
rapid_games[0].headers

Headers(Event='Live Chess', Site='Chess.com', Date='2022.06.30', Round='-', White='uneatenbean', Black='lucaippolito', Result='0-1', BlackElo='1268', CurrentPosition='8/8/p7/8/P7/1P3p2/4K1k1/8 w - -', ECO='D35', ECOUrl='https://www.chess.com/openings/Queens-Gambit-Declined-Exchange-Positional-Line-5...Be7-6.Nf3', EndDate='2022.06.30', EndTime='12:13:17', Link='https://www.chess.com/game/live/50329172097', StartTime='11:56:05', Termination='lucaippolito won by resignation', TimeControl='600', Timezone='UTC', UTCDate='2022.06.30', UTCTime='11:56:05', WhiteElo='1282')

In [52]:
ratings = [g.headers["WhiteElo"] if g.headers["White"] == USERNAME else g.headers["BlackElo"] for g in rapid_games]

In [53]:
ratings

['1282',
 '1291',
 '1298',
 '1306',
 '1314',
 '1325',
 '1319',
 '1330',
 '1324',
 '1315',
 '1307',
 '1316',
 '1307',
 '1299',
 '1309',
 '1302',
 '1311']

### Date
Date is in YYYY.MM.DD format, I will read into a date property after converting to epoch then average rating per day in Pandas Dataframe

In [61]:
from datetime import datetime
import pytz

In [62]:
def parse_date(date: str) -> int:
    return datetime.strptime(date, "%Y.%m.%d").replace(tzinfo=pytz.UTC).timestamp()

assert parse_date('2022.06.30') == 1656547200.0

1656547200.0


In [70]:
rating_times = [[int(g.headers["WhiteElo"] if g.headers["White"] == USERNAME else g.headers["BlackElo"]), parse_date(g.headers["UTCDate"])] for g in rapid_games]
rating_times[0]

[1282, 1656547200.0]

In [71]:
df = pd.DataFrame(rating_times, columns=["Ratings", "Date"])
df.describe()

,Ratings,Date
count,17.000000,1.700000e+01
mean,1309.117647,1.655917e+09
std,12.439264,8.143886e+05
min,1282.000000,1.654301e+09
25%,1302.000000,1.655942e+09
50%,1309.000000,1.655942e+09
75%,1316.000000,1.656461e+09
max,1330.000000,1.656547e+09


In [72]:
df.groupby(['Date']).mean()

,Ratings
Date,
1.654301e+09,1307.333333
1.655942e+09,1311.333333
1.656461e+09,1322.000000
1.656547e+09,1294.250000
